## K-近邻算法（kNN）

**原理**：通过测量不同特征值之间距离的方法进行分类

**优点**：精度高、对异常值不敏感、无数据输入假定
**缺点**：计算复杂度高，空间复杂度高
**适应数据**：数值型、标称型

### 一、算法描述
算法（K-近邻法）

**输入**：训练数据集

$$ T=\{(x_1,y_1),(x_2,y_2),\cdots,(x_N,y_N)\} $$

其中，$x_i\in\chi\subseteq R^n$为实例的特征向量，$y_i\in\gamma=\{c_1,c_2,\cdots,c_K\}$为实例的类别，$i=1,2,\cdots.,N$；实例特征向量$x$

**输出**：实例$x$所属的类$y$

（1）根据给定的距离度量，在训练集$T$中找出与$x$最邻近的$k$个点，涵盖这$k$个点的$x$的邻域记作$N_k(x)$；

（2）在$N_k(x)$中根据分类决策规则（如多数表决）决定$x$的类别$y$：

$$y=\underset{c_j}{argmax}\sum_{x_i\in N_k(x)}I(y_i=c_j),\; i=1,2,\cdots,N;\; j=1,2,\cdots,K \;\;\;(\star)$$

式$(*)$中，$I$为指示函数，即当$y_i=c_j$时$I$为1，否则$I$为0.
    $k$近邻法的特殊情况是$k=1$的情形，称为最近邻算法，对于输入的实例点（特征向量）$x$，最近邻法将训练数据集中与$x$最近邻点的类作为$x$的类.
        
根据算法可知，**k近邻主要的3个问题**：

（1）度量距离的计算

（2）计算出距离排序后选择k个点，k值如何确定

（3）选出k个点后如何进行分类决策

### 二、距离度量
需要计算未知样本与各个训练样本的**距离**（实际上是反应两个实例点的相似程度）。

设两个点$x_i=(x_i^{(1)},x_i^{(2)},\cdots,x_i^{(n)}),x_j=(x_j^{(1)},x_j^{(2)},\cdots,x_j^{(n)})$，n表示特征数，距离计算公式：

$L_p(x_i,x_j)=(\sum_{l=1}^{n}|x_i^{(l)}-x_j^{(l)}|^p)^\frac{1}{p}$，其中$p\geq1$

①欧式距离（p=2）

$L_2(x_i,x_j)=(\sum_{l=1}^{n}|x_i^{(l)}-x_j^{(l)}|^2)^\frac{1}{2}$

②曼哈顿距离（p=1）

$L_1(x_i,x_j)=\sum_{l=1}^{n}|x_i^{(l)}-x_j^{(l)}|$

③坐标距离最大值（p=∞）

$L_∞(x_i,x_j)=\max_l |x_i^{(l)}-x_j^{(l)}|$

### 三、k值的选择
近似误差与估计误差的区别：
```
近似误差可以理解为对训练集的训练误差，它只能体现对训练数据的拟合表现；
估计误差可以理解为对测试集的测试误差，它可以体现对测试数据的拟合表现，也就是体现出泛化能力，因此对模型要求是估计误差越小越好。
```
如果k值过大，那么近似误差会增大，可能会减少估计误差单对预测也没什么效果。实际应用中，k值一般取一个比较小的数值，通过交叉验证法来选取最优的k值。

### 四、分类决策规则
一般是多数表决

### 五、代码实现
classify0函数的功能是使用k-近邻算法将每组数据划分到某个类中，伪代码如下：

对未知类别属性的数据集中的每个点依次执行以下操作：

（1）计算已知类别数据集中的点与当前点之间的距离；

（2）按照距离递增次序排序；

（3）选取与当前点距离最小的k个点；

（4）确定前k个点所在类别的出现频率；

（5）返回前k个点出现频率最高的类别作为当前点的预测分类。

In [ ]:
from numpy import *
import operator

# k近邻分类算法
def classify0(inX, dataSet, labels, k):
    '''
      inX：用来分类的测试集
      dataSet：用来训练的数据集
      labels：用来训练的数据集的标签
      k：k近邻的k值
    '''
    #欧式距离计算
    dataSetSize = dataSet.shape[0]
    diffMat = tile(inX, (dataSetSize,1)) - dataSet  # tile()：将矩阵inX重复，行方向上重复dataSetSize次，列方向上重复1次
    sqDiffMat = diffMat**2
    sqDistinces = sqDiffMat.sum(axis=1)  #行方向求和
    distances = sqDistinces**0.5
    sortedDistIndicies = distances.argsort() #将距离值进行排序，并返回索引值
    
    #选择距离最小的k个点，并统计k个点中各个类别的数目
    classCount = {}
    for i in range(k):
        voteIlabel = labels[sortedDistIndicies[i]]
        classCount[voteIlabel] = classCount.get(voteIlabel,0) + 1  #get(key,default)：若键key在字典中，返回key的值，否则返回默认值
    
    #排序，选择类别数目最多的所属类别
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    
    return sortedClassCount[0][0]


#归一化数值
def autoNorm(dataSet):
    minVals = dataSet.min(0)
    maxVals = dataSet.max(0)
    ranges = maxVals - minVals
    normDataSet = zeros(shape(dataSet))
    m = dataSet.shape[0]  #获取数据行数
    normDataSet = dataSet - tile(minVals, (m,1))
    normDataSet = normDataSet/tile(ranges, (m,1))
    return normDataSet, ranges, minVals


#划分训练集/测试集数据进行测试
def datingClassTest(datingDataMat, datingLabels):
    hoRatio = 0.1  #测试集所占比例
    normMat, ranges, minVals = autoNorm(datingDataMat)  #归一化处理
    m = normMat.shape[0]
    numTestVecs = int(m*hoRatio)  #得到测试集个数
    errorCount = 0  #记录错误的个数
    for i in range(numTestVecs):
        classifierResult = classify0(normMat[i,:], normMat[numTestVecs:m,:], datingLabels[numTestVecs:m], 3)
        print("预测结果：%d， 真实标签：%d"%(classifierResult, datingLabels[i]))
        if(classifierResult != datingLabels[i]):
            errorCount += 1
    print("测试集错误率为：%f"%(errorCount/float(numTestVecs)))    

科学计算包：numpy
运算符模块：operator

基本通用函数：

In [3]:
def createDataSet():
    group = array([[1.0,1,1],[1.0,1.0],[0,0],[0,0.1]])
    labels = ['A','A','B','B']
    return group, labels

In [4]:
import kNN

ModuleNotFoundError: No module named 'kNN'